In [1]:
import numpy as np
import matplotlib.pyplot as plt

from units import *
from profiles import Profiles

%load_ext autoreload
%autoreload 2

# A toy simulation

In [2]:
class DemoSim:
    def __init__(self, theta_x_lims=[-1.6, 1.6], theta_y_lims=[-0.9, 0.9]):
        """
        Class to create animations of astrometric weak lensing. For demo and not science!
        :param theta_x_lims: x-axis coordinate limits [x_min, x_max] in arcsecs
        :param theta_y_lims: y-axis coordinate limits [y_min, y_max] in arcsecs
        """

        self.theta_x_lims = theta_x_lims
        self.theta_y_lims = theta_y_lims

        # Area of sky region
        self.roi_area = (self.theta_x_lims[1] - self.theta_x_lims[0]) * \
                        (self.theta_y_lims[1] - self.theta_y_lims[0])

    def animation(self, pos_l, M_l, R_l, v_l, D_l,
                  n_sources_fix=True,
                  # Source properties
                  std_mu_s=1e-6,
                  n_dens=20, source_pos="random"
                  ):
        """
        :param pos_l: tuple of lens positions, format [[x_1, y_1], [x_2, y_2]...]
        :param M_l: tuple of lens masses
        :param R_l: tuple of lens sizes (Gaussian lens)
        :param v_l: tuple of lens velocities
        :param D_l: tuple of distances of lenses
        :param n_dens: density of sources (per arcsecs^2); default 20
        :param source_pos: must be one of ["uniform", "random"]; default "random"
        """

        # Get total number of sources in sky region
        
        if n_sources_fix:
            self.n_total = np.floor(n_dens * self.roi_area).astype(np.int32)
        else:
            self.n_total = np.random.poisson(n_dens * self.roi_area)
        
        # Source velocities
        self.mu_s_intrinsic = np.random.normal(loc=0, scale=std_mu_s, size=(ds.n_total, 2))
        
        # Set source positions

        # Random positions + custom if specified
        if source_pos == "random":

            # Initial source property array
            self.sources = np.zeros(self.n_total, dtype=[("theta_x", float, 1),
                                                         ("theta_y", float, 1),
                                                         ("theta_x_0", float, 1),
                                                         ("theta_y_0", float, 1),
                                                         ("mu", float, 1)])

            self.sources["theta_x_0"] = np.array(
                list(np.random.uniform(*self.theta_x_lims, self.n_total)))
            self.sources["theta_y_0"] = np.array(
                list(np.random.uniform(*self.theta_x_lims, self.n_total)))

        # Uniform grid of sources
        elif source_pos == "uniform":
            xy_ratio = (self.theta_y_lims[1] - self.theta_y_lims[0]) / \
                (self.theta_x_lims[1] - self.theta_x_lims[0])
            x_pos = np.linspace(self.theta_x_lims[0], self.theta_x_lims[1], np.round(
                np.sqrt(self.n_total / xy_ratio)))
            y_pos = np.linspace(self.theta_y_lims[0], self.theta_y_lims[1], np.round(
                np.sqrt(self.n_total * xy_ratio)))

            self.n_total = len(np.meshgrid(x_pos, y_pos)[0].flatten())

            # Initialize source property array
            self.sources = np.zeros(self.n_total, dtype=[("theta_x", float, 1),
                                                         ("theta_y", float, 1),
                                                         ("theta_x_0", float, 1),
                                                         ("theta_y_0", float, 1)])

            self.sources["theta_x_0"] = np.meshgrid(x_pos, y_pos)[0].flatten()
            self.sources["theta_y_0"] = np.meshgrid(x_pos, y_pos)[1].flatten()

        assert len(pos_l) == len(v_l) == len(M_l) == len(R_l) == len(D_l), \
            "Lens property arrays must be the same size!"

        # Infer number of lenses
        self.n_lens = len(pos_l)

        # Initialize lens property array
        self.lenses = np.zeros(self.n_lens, dtype=[("theta_x", float, 1),
                                                   ("theta_y", float, 1),
                                                   ("M_0", float, 1),
                                                   ("R_0", float, 1),
                                                   ("D", float, 1),
                                                   ("v_x", float, 1),
                                                   ("v_y", float, 1)])

        # Set initial source positions
        self.sources["theta_x"] = self.sources["theta_x_0"]
        self.sources["theta_y"] = self.sources["theta_y_0"]

        # Set initial lens positions...
        self.lenses["theta_x"] = np.array(pos_l)[:, 0]
        self.lenses["theta_y"] = np.array(pos_l)[:, 1]

        # ... and lens properties
        self.lenses["v_x"] = np.array(v_l)[:, 0]
        self.lenses["v_y"] = np.array(v_l)[:, 1]
        self.lenses["M_0"] = np.array(M_l)
        self.lenses["R_0"] = np.array(R_l)
        self.lenses["D"] = np.array(D_l)

        theta_s = np.zeros((self.n_total, 2))

        # Deflection and proper motion vectors
        for i_lens in range(self.n_lens):
            b_ary = np.transpose([self.sources["theta_x"] - self.lenses["theta_x"][i_lens],
                                  self.sources["theta_y"] - self.lenses["theta_y"][i_lens]]) * asctorad

            vel_l = np.array([self.lenses["v_x"][i_lens],
                              self.lenses["v_y"][i_lens]])
            
            for i_source in range(self.n_total):

                theta_s[i_source] += self.theta(b_ary[i_source], self.lenses["R_0"][i_lens],
                                                self.lenses["M_0"][i_lens], self.lenses["D"][i_lens])
                        
        # New source positions including deflection
        self.sources["theta_x"] = self.sources["theta_x_0"] + theta_s[:, 0]
        self.sources["theta_y"] = self.sources["theta_y_0"] + theta_s[:, 1]

    def update(self, dt):
        """ Update animation """

        theta_s = np.zeros((self.n_total, 2))

        for i_lens in range(self.n_lens):
            
            b_ary = np.transpose([self.sources["theta_x"] - self.lenses["theta_x"][i_lens],
                                  self.sources["theta_y"] - self.lenses["theta_y"][i_lens]]) * asctorad

            vel_l = np.array([self.lenses["v_x"][i_lens],
                              self.lenses["v_y"][i_lens]])
            
            for i_source in range(self.n_total):    
                
                theta_s[i_source] += self.theta(b_ary[i_source], self.lenses["R_0"][i_lens],
                                                self.lenses["M_0"][i_lens], self.lenses["D"][i_lens])
 
            mu_l = (vel_l / self.lenses["D"][i_lens]) / (Year ** -1) * radtoasc
            
            self.lenses["theta_x"][i_lens] = self.lenses["theta_x"][i_lens] + mu_l[0] * dt
            self.lenses["theta_y"][i_lens] = self.lenses["theta_y"][i_lens] + mu_l[1] * dt
                    
        # Update intrinsic source position
        self.sources["theta_x_0"] += self.mu_s_intrinsic[:, 0] * dt
        self.sources["theta_y_0"] += self.mu_s_intrinsic[:, 1] * dt
                
        self.sources["theta_x"] = self.sources["theta_x_0"] + theta_s[:, 0] 
        self.sources["theta_y"] = self.sources["theta_y_0"] + theta_s[:, 1]


    @classmethod
    def mu(self, beta_vec, v_ang_vec, R_0, M_0, d_lens):
        """ Get lens-induced proper motion vector
        """

        # Convert angular to physical impact parameter
        b_vec = d_lens * np.array(beta_vec)
        # Convert angular to physical velocity
        v_vec = d_lens * np.array(v_ang_vec)
        b = np.linalg.norm(b_vec)  # Impact parameter
        M, dMdb, _ = Profiles.MdMdb_Gauss(b, R_0, M_0)
        b_unit_vec = b_vec / b  # Convert angular to physical impact parameter
        b_dot_v = np.dot(b_unit_vec, v_vec)
        factor = (dMdb / b * b_unit_vec * b_dot_v
                  + M / b ** 2 * (v_vec - 2 * b_unit_vec * b_dot_v))

        return -factor * 4 * GN / (asctorad / Year)  # Convert to as/yr

    @classmethod
    def theta(self, beta_vec, R_0, M_0, d_lens):
        """ Get lens-induced deflection vector
        """

        # Convert angular to physical impact parameter
        b_vec = d_lens * np.array(beta_vec)
        b = np.linalg.norm(b_vec)  # Impact parameter
        M, _, _ = Profiles.MdMdb_Gauss(b, R_0, M_0)
        b_unit_vec = b_vec / b  # Convert angular to physical impact parameter

        return 4 * GN * M / b * b_unit_vec * radtoasc  # Convert to as

In [3]:
from tqdm.notebook import tqdm

In [4]:
ds = DemoSim(theta_x_lims = [-0.1, 0.1], theta_y_lims = [-0.1, 0.1])
ds.animation(pos_l=np.array([[0.,0]]), v_l=np.array([[200, 0]]) * Kmps, R_l=[10 * pc], M_l=[0],
                        D_l=[1 * kpc],
                        n_dens=500,
                        std_mu_s=1e-6,
                        source_pos="random")

/tmp/ipykernel_258448/4169467666.py:48: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  self.sources = np.zeros(self.n_total, dtype=[("theta_x", float, 1),
/tmp/ipykernel_258448/4169467666.py:86: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  self.lenses = np.zeros(self.n_lens, dtype=[("theta_x", float, 1),


In [28]:
n_sim = 10000

x = np.zeros((n_sim, ds.n_total, 10, 2))
y = np.zeros((n_sim))

for i_sim in tqdm(range(n_sim)):
    
    ds = DemoSim(theta_x_lims = [-0.1, 0.1], theta_y_lims = [-0.1, 0.1])
    
    y_lens = np.random.randint(2)
    y[i_sim] = y_lens
    
    if y_lens:
        M_l = 10 ** 7. * M_s
    else:
        M_l = 0
    
    pos_l = 0.1 * (np.random.random(2) - 0.5)
    v_l = 300. * (np.random.random(2) - 0.5)
    
    anim = ds.animation(pos_l=np.array([pos_l]), v_l=np.array([v_l]) * Kmps, R_l=[1 * pc], M_l=[M_l],
                        D_l=[1 * kpc],
                        n_dens=500,
                        std_mu_s=2e-5,
                        source_pos="random")

    for i in range(10):
        
        ds.update(dt=10)
        
        x[i_sim, : , i, 0] = ds.sources["theta_x"]
        x[i_sim, : , i, 1] = ds.sources["theta_y"]

  0%|          | 0/10000 [00:00<?, ?it/s]

/tmp/ipykernel_258448/4169467666.py:48: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  self.sources = np.zeros(self.n_total, dtype=[("theta_x", float, 1),
/tmp/ipykernel_258448/4169467666.py:86: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  self.lenses = np.zeros(self.n_lens, dtype=[("theta_x", float, 1),


In [29]:
error_mu = 2e-5
x_err = np.random.normal(loc=0, scale=error_mu, size=(x.shape))

x += x_err
x = torch.Tensor(x)

# Time-series deep set model

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

In [31]:
from einops import rearrange

In [32]:
# Normalize and rotate data
x -= x[:, :, 0, :].unsqueeze(-2)
theta = torch.atan(x[:, :, -1, 1] / x[:, :, -1, 0]).numpy()
rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
torch.Tensor(rot)
rot = torch.Tensor(rearrange(rot, 'i j k l  -> k l i j'))
x = torch.matmul(x, rot)

In [33]:
class DeepSetLSTM(nn.Module):
    def __init__(self, num_outputs=1, dim_output=1, dim_hidden=128):
        super(DeepSetLSTM, self).__init__()
        
        self.num_outputs = num_outputs
        self.dim_output = dim_output
        self.dim_hidden = dim_hidden
        
        self.phi = nn.LSTM(input_size=2, hidden_size=dim_hidden, num_layers=8, batch_first=True, bidirectional=True)
        
        self.rho = nn.Sequential(
                nn.Linear(2 * dim_hidden, dim_hidden),
                nn.ReLU(),
                nn.Linear(dim_hidden, dim_hidden),
                nn.ReLU(),
                nn.Linear(dim_hidden, dim_hidden),
                nn.ReLU(),
                nn.Linear(dim_hidden, dim_hidden),
                nn.ReLU(),
                nn.Linear(dim_hidden, dim_hidden),
                nn.ReLU(),
                nn.Linear(dim_hidden, dim_hidden),
                nn.ReLU(),
                nn.Linear(dim_hidden, num_outputs*dim_output))

    def forward(self, x):
        batch_size = x.shape[0]
        x = rearrange(x, 'b s t xy -> (b s) t xy')
        x, (h, c) = self.phi(x) 
        h = rearrange(h, '(d l) b h -> b d l h', d=2)[:, :, -1, :]
        h = rearrange(h, 'b d h -> b (d h)')
        
        h = rearrange(h, '(b s) h -> b s h', b=batch_size).mean(-2)
        
        x = self.rho(h).reshape(-1, self.num_outputs, self.dim_output)
        
        return x

In [34]:
from torch.utils.data import TensorDataset, DataLoader, random_split, SubsetRandomSampler

import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, EarlyStopping

In [35]:
y = torch.Tensor(y)

In [36]:
# Standardize
x = (x - x.mean(0).mean(0)) / x.std(0).std(0)
x[:, :, 0, :] = 0.

In [37]:
val_fraction = 0.1
n_samples_val = int(val_fraction * len(x))

dataset = TensorDataset(x, y)

dataset_train, dataset_val = random_split(dataset, [len(x) - n_samples_val, n_samples_val])
train_loader = DataLoader(dataset_train, batch_size=64, num_workers=8, pin_memory=True, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=64, num_workers=8, pin_memory=True, shuffle=False)

In [38]:
class MultiBlipClassifier(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.deepset = DeepSetLSTM()

    def configure_optimizers(self):
        
        optimizer = optim.AdamW(self.parameters(), lr=5e-3, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=4)
        
        return {"optimizer": optimizer, 
                    "lr_scheduler": {
                    "scheduler": scheduler,
                    "interval": "epoch",
                    "monitor": "val_loss",
                    "frequency": 1}
                }
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.deepset(x)
        loss = nn.BCEWithLogitsLoss(reduction='mean')(y_hat[:,0,0], y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.deepset(x)
        loss = nn.BCEWithLogitsLoss(reduction='mean')(y_hat[:,0,0], y)
        self.log('val_loss', loss)

# Train model

In [39]:
model = MultiBlipClassifier()

In [40]:
checkpoint_callback = ModelCheckpoint(monitor="val_loss")
early_stop_callback = EarlyStopping(monitor='val_loss', patience=8)        
lr_monitor = LearningRateMonitor(logging_interval='epoch')

trainer = pl.Trainer(max_epochs=25, gpus=1, gradient_clip_val=1., callbacks=[checkpoint_callback, early_stop_callback, lr_monitor])
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)
model.load_from_checkpoint(checkpoint_callback.best_model_path);

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name    | Type        | Params
----------------------------------------
0 | deepset | DeepSetLSTM | 3.0 M 
----------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
12.070    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Traceback (most recent call last):
  File "/n/home11/smsharma/.conda/envs/deepsets/lib/python3.9/multiprocessing/queues.py", line 251, in _feed
    send_bytes(obj)
  File "/n/home11/smsharma/.conda/envs/deepsets/lib/python3.9/multiprocessing/connection.py", line 205, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/n/home11/smsharma/.conda/envs/deepsets/lib/python3.9/multiprocessing/connection.py", line 416, in _send_bytes
    self._send(header + buf)
  File "/n/home11/smsharma/.conda/envs/deepsets/lib/python3.9/multiprocessing/connection.py", line 373, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]